In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO



In [63]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

q = '''
SELECT trim(k.keyword) as keyword,
       k.bid_price,
       p.id as ad_set_product_id,
       a.id as ad_set_id
FROM ad_set a
JOIN ad_set_product p on a.id = p.ad_set_id
JOIN ad_set_product_bid_keyword k ON k.ad_set_product_id = p.id
WHERE a.status = 3
'''
ad_set = pd.read_sql(q, con=mysql)

q = '''
select if(@grouping = tmp.keyword, @rank:=@rank+1, @rank:=1) AS 'rank',
       @grouping:=tmp.keyword as 'keyword',
       tmp.bid_price AS 'bid_price',
       tmp.count
from
(select trim(apk.keyword) AS keyword, apk.bid_price, count(*) AS 'count' from
        ad_set ad
        join ad_set_product ap on ad.id = ap.ad_set_id
        join ad_set_product_bid_keyword apk on apk.ad_set_product_id = ap.id
        where ad.status = 3
          and ap.status = 3
          and apk.active = 1
        group by 1, 2
            order by 1 asc, 2 desc) as tmp,
(select @rank:=0, @grouping:='') as b
group by tmp.keyword, tmp.bid_price
'''
bid_keyword_rank = pd.read_sql(q, con=mysql)

Current Time = 15:57:09


In [64]:
ad_set['bid_price'] = ad_set['bid_price'].astype(int)
bid_keyword_rank['bid_price'] = bid_keyword_rank['bid_price'].astype(int)
bid_keyword_rank['rank'] = bid_keyword_rank['rank'].astype(float)

In [68]:
ad = pd.merge(ad_set, bid_keyword_rank, on=['keyword', 'bid_price'])

In [71]:
pre_mean = ad.groupby('ad_set_id', as_index=False)['rank'].mean()

In [73]:
pre_mean.to_csv('pre_mean.csv', index=False)

In [72]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

q = '''
SELECT trim(k.keyword) as keyword,
       k.bid_price,
       p.id as ad_set_product_id,
       a.id as ad_set_id
FROM ad_set a
JOIN ad_set_product p on a.id = p.ad_set_id
JOIN ad_set_product_bid_keyword k ON k.ad_set_product_id = p.id
WHERE a.status = 3
'''
ad_set2 = pd.read_sql(q, con=mysql)
print('Done getting ad set data')
q = '''
select if(@grouping = tmp.keyword, @rank:=@rank+1, @rank:=1) AS 'rank',
       @grouping:=tmp.keyword as 'keyword',
       tmp.bid_price AS 'bid_price',
       tmp.count
from
(select trim(apk.keyword) AS keyword, apk.bid_price, count(*) AS 'count' from
        ad_set ad
        join ad_set_product ap on ad.id = ap.ad_set_id
        join ad_set_product_bid_keyword apk on apk.ad_set_product_id = ap.id
        where ad.status = 3
          and ap.status = 3
          and apk.active = 1
        group by 1, 2
            order by 1 asc, 2 desc) as tmp,
(select @rank:=0, @grouping:='') as b
group by tmp.keyword, tmp.bid_price
'''
bid_keyword_rank2 = pd.read_sql(q, con=mysql)
print('Done getting bid keyword rank data')

ad_set2['bid_price'] = ad_set2['bid_price'].astype(int)
bid_keyword_rank2['bid_price'] = bid_keyword_rank2['bid_price'].astype(int)
bid_keyword_rank2['rank'] = bid_keyword_rank2['rank'].astype(float)

ad2 = pd.merge(ad_set2, bid_keyword_rank2, on=['keyword', 'bid_price'])
post_30m_mean = ad2.groupby('ad_set_id', as_index=False)['rank'].mean()

Current Time = 16:29:50
Done getting ad set data
Done getting bid keyword rank data


In [75]:
post_30m_mean.to_csv('post_30m_mean', index=False)

In [78]:
compare_30m = pd.merge(pre_mean, post_30m_mean, on='ad_set_id')
compare_30m['diff'] = compare_30m['rank_x'] - compare_30m['rank_y']

In [80]:
compare_30m.describe()

,ad_set_id,rank_x,rank_y,diff
count,1922.000000,1922.000000,1922.000000,1922.000000
mean,79292.151405,2.279978,2.305743,-0.025765
std,2660.474874,1.855200,1.824749,0.185123
min,63454.000000,1.000000,1.000000,-1.000000
25%,78828.750000,1.166667,1.200000,0.000000
50%,80047.500000,1.600000,1.666667,0.000000
75%,80708.500000,2.500000,2.600000,0.000000
max,81302.000000,16.000000,13.666667,2.333333


In [90]:
pd.merge(ad2.groupby('ad_set_id', as_index=False)['keyword'].nunique(), compare_30m, on='ad_set_id').to_csv('./ad2.csv', index=False)

In [91]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

q = '''
SELECT trim(k.keyword) as keyword,
       k.bid_price,
       p.id as ad_set_product_id,
       a.id as ad_set_id
FROM ad_set a
JOIN ad_set_product p on a.id = p.ad_set_id
JOIN ad_set_product_bid_keyword k ON k.ad_set_product_id = p.id
WHERE a.status = 3
'''
ad_set3 = pd.read_sql(q, con=mysql)
print('Done getting ad set data')
q = '''
select if(@grouping = tmp.keyword, @rank:=@rank+1, @rank:=1) AS 'rank',
       @grouping:=tmp.keyword as 'keyword',
       tmp.bid_price AS 'bid_price',
       tmp.count
from
(select trim(apk.keyword) AS keyword, apk.bid_price, count(*) AS 'count' from
        ad_set ad
        join ad_set_product ap on ad.id = ap.ad_set_id
        join ad_set_product_bid_keyword apk on apk.ad_set_product_id = ap.id
        where ad.status = 3
          and ap.status = 3
          and apk.active = 1
        group by 1, 2
            order by 1 asc, 2 desc) as tmp,
(select @rank:=0, @grouping:='') as b
group by tmp.keyword, tmp.bid_price
'''
bid_keyword_rank3 = pd.read_sql(q, con=mysql)
print('Done getting bid keyword rank data')

ad_set3['bid_price'] = ad_set3['bid_price'].astype(int)
bid_keyword_rank3['bid_price'] = bid_keyword_rank3['bid_price'].astype(int)
bid_keyword_rank3['rank'] = bid_keyword_rank3['rank'].astype(float)

ad3 = pd.merge(ad_set3, bid_keyword_rank3, on=['keyword', 'bid_price'])
post_1hr_mean = ad3.groupby('ad_set_id', as_index=False)['rank'].mean()

Current Time = 16:59:09
Done getting ad set data
Done getting bid keyword rank data


In [92]:
compare_1hr = pd.merge(pre_mean, post_1hr_mean, on='ad_set_id')
compare_1hr['diff'] = compare_1hr['rank_x'] - compare_1hr['rank_y']

In [93]:
compare_1hr.describe()

,ad_set_id,rank_x,rank_y,diff
count,1916.000000,1916.000000,1916.000000,1916.000000
mean,79288.102296,2.282551,2.307889,-0.025338
std,2663.601742,1.857350,1.826632,0.186827
min,63454.000000,1.000000,1.000000,-1.000000
25%,78825.250000,1.166667,1.200000,0.000000
50%,80043.500000,1.600000,1.666667,0.000000
75%,80707.500000,2.500000,2.600000,0.000000
max,81302.000000,16.000000,13.666667,2.333333


In [94]:

pd.merge(ad3.groupby('ad_set_id', as_index=False)['keyword'].nunique(), compare_1hr, on='ad_set_id').to_csv('./ad3.csv', index=False)


In [95]:
compare_1hr

,ad_set_id,rank_x,rank_y,diff
0,63454,6.280000,5.760000,0.520000
1,64002,11.657658,10.603604,1.054054
2,64083,2.500000,2.500000,0.000000
3,64113,1.400000,1.600000,-0.200000
4,64209,3.666667,3.625000,0.041667
...,...,...,...,...
1911,81296,1.166667,1.333333,-0.166667
1912,81297,1.000000,1.000000,0.000000
1913,81298,1.250000,1.375000,-0.125000
1914,81301,1.666667,1.666667,0.000000


In [96]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

q = '''
SELECT trim(k.keyword) as keyword,
       k.bid_price,
       p.id as ad_set_product_id,
       a.id as ad_set_id
FROM ad_set a
JOIN ad_set_product p on a.id = p.ad_set_id
JOIN ad_set_product_bid_keyword k ON k.ad_set_product_id = p.id
WHERE a.status = 3
'''
ad_set4 = pd.read_sql(q, con=mysql)
print('Done getting ad set data')

q = '''
select if(@grouping = tmp.keyword, @rank:=@rank+1, @rank:=1) AS 'rank',
       @grouping:=tmp.keyword as 'keyword',
       tmp.bid_price AS 'bid_price',
       tmp.count
from
(select trim(apk.keyword) AS keyword, apk.bid_price, count(*) AS 'count' from
        ad_set ad
        join ad_set_product ap on ad.id = ap.ad_set_id
        join ad_set_product_bid_keyword apk on apk.ad_set_product_id = ap.id
        where ad.status = 3
          and ap.status = 3
          and apk.active = 1
        group by 1, 2
            order by 1 asc, 2 desc) as tmp,
(select @rank:=0, @grouping:='') as b
group by tmp.keyword, tmp.bid_price
'''
bid_keyword_rank4 = pd.read_sql(q, con=mysql)
print('Done getting bid keyword rank data')

ad_set4['bid_price'] = ad_set4['bid_price'].astype(int)
bid_keyword_rank4['bid_price'] = bid_keyword_rank4['bid_price'].astype(int)
bid_keyword_rank4['rank'] = bid_keyword_rank4['rank'].astype(float)

ad4 = pd.merge(ad_set4, bid_keyword_rank4, on=['keyword', 'bid_price'])
post_2hr_mean = ad4.groupby('ad_set_id', as_index=False)['rank'].mean()

Current Time = 17:58:52
Done getting ad set data
Done getting bid keyword rank data


In [97]:
compare_2hr = pd.merge(pre_mean, post_2hr_mean, on='ad_set_id')
compare_2hr['diff'] = compare_2hr['rank_x'] - compare_2hr['rank_y']

In [98]:
compare_2hr.describe()

,ad_set_id,rank_x,rank_y,diff
count,1834.000000,1834.000000,1834.000000,1834.000000
mean,79330.653217,2.283368,2.299571,-0.016202
std,2625.503584,1.875917,1.816163,0.227019
min,63454.000000,1.000000,1.000000,-2.000000
25%,78893.500000,1.150000,1.200000,0.000000
50%,80048.500000,1.571429,1.666667,0.000000
75%,80732.250000,2.500000,2.586039,0.000000
max,81302.000000,16.000000,14.666667,2.083333


In [100]:
compare_2hr.sort_values('diff')

,ad_set_id,rank_x,rank_y,diff
472,78936,1.000000,3.000000,-2.000000
574,79261,2.000000,3.000000,-1.000000
1362,80713,2.000000,3.000000,-1.000000
1335,80682,1.000000,2.000000,-1.000000
1806,81270,1.000000,2.000000,-1.000000
...,...,...,...,...
17,65414,8.793103,7.505747,1.287356
297,78411,16.000000,14.666667,1.333333
983,80167,9.857143,8.428571,1.428571
922,80054,12.000000,10.000000,2.000000


In [118]:
test = pd.merge(ad[['keyword', 'bid_price', 'rank', 'ad_set_product_id', 'ad_set_id']], ad4[['keyword', 'bid_price', 'rank', 'ad_set_product_id', 'ad_set_id']], on=['keyword', 'bid_price', 'ad_set_id', 'ad_set_product_id'])
test['diff_by_keyword'] = test['rank_x'] - test['rank_y']

In [123]:
test.sort_values('diff_by_keyword')

,keyword,bid_price,rank_x,ad_set_product_id,ad_set_id,rank_y,diff_by_keyword
11660,컨버스,105,5.0,220109,80376,7.0,-2.0
1094,컨버스,110,4.0,220243,80457,6.0,-2.0
6311,컨버스,125,1.0,221382,81108,3.0,-2.0
6312,컨버스,125,1.0,221572,81254,3.0,-2.0
6313,235,125,1.0,221382,81108,3.0,-2.0
...,...,...,...,...,...,...,...
4661,컴퓨터,50,41.0,215806,78430,37.0,4.0
14902,컴퓨터,65,39.0,180987,64002,35.0,4.0
4662,컴퓨터,50,41.0,215807,78430,37.0,4.0
4658,컴퓨터,50,41.0,215803,78430,37.0,4.0


In [99]:

pd.merge(ad4.groupby('ad_set_id', as_index=False)['keyword'].nunique(), compare_2hr, on='ad_set_id').to_csv('./ad4.csv', index=False)


In [124]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

q = '''
SELECT trim(k.keyword) as keyword,
       k.bid_price,
       p.id as ad_set_product_id,
       a.id as ad_set_id
FROM ad_set a
JOIN ad_set_product p on a.id = p.ad_set_id
JOIN ad_set_product_bid_keyword k ON k.ad_set_product_id = p.id
WHERE a.status = 3
'''
ad_set5 = pd.read_sql(q, con=mysql)
print('Done getting ad set data')

q = '''
select if(@grouping = tmp.keyword, @rank:=@rank+1, @rank:=1) AS 'rank',
       @grouping:=tmp.keyword as 'keyword',
       tmp.bid_price AS 'bid_price',
       tmp.count
from
(select trim(apk.keyword) AS keyword, apk.bid_price, count(*) AS 'count' from
        ad_set ad
        join ad_set_product ap on ad.id = ap.ad_set_id
        join ad_set_product_bid_keyword apk on apk.ad_set_product_id = ap.id
        where ad.status = 3
          and ap.status = 3
          and apk.active = 1
        group by 1, 2
            order by 1 asc, 2 desc) as tmp,
(select @rank:=0, @grouping:='') as b
group by tmp.keyword, tmp.bid_price
'''
bid_keyword_rank5 = pd.read_sql(q, con=mysql)
print('Done getting bid keyword rank data')

ad_set5['bid_price'] = ad_set5['bid_price'].astype(int)
bid_keyword_rank5['bid_price'] = bid_keyword_rank5['bid_price'].astype(int)
bid_keyword_rank5['rank'] = bid_keyword_rank5['rank'].astype(float)

ad5 = pd.merge(ad_set5, bid_keyword_rank5, on=['keyword', 'bid_price'])
post_3hr_mean = ad5.groupby('ad_set_id', as_index=False)['rank'].mean()

Current Time = 18:59:57
Done getting ad set data
Done getting bid keyword rank data


In [171]:
a['cum_rank'] = cum_rank

In [172]:
a

,rank,keyword,bid_price,count,cum_rank
1,1.0,.,55,1,1
0,2.0,.,50,1,2
2,1.0,0w30,50,1,1
3,1.0,0w40,50,1,1
7,1.0,100,195,1,1
...,...,...,...,...,...
15420,2.0,힙색,55,1,2
15422,1.0,힙색크로스백,50,1,1
15423,1.0,힙색크로스백가방,50,1,1
15424,1.0,힙쎅,50,1,1


In [161]:
com = a[a['keyword'] == '컴퓨터']
com['cumsum'] = com['count'].cumsum()
com

/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,rank,keyword,bid_price,count,cumsum
13266,1.0,컴퓨터,1000,1,1
13265,2.0,컴퓨터,950,2,3
13264,3.0,컴퓨터,910,1,4
13263,4.0,컴퓨터,900,1,5
13262,5.0,컴퓨터,895,1,6
13261,6.0,컴퓨터,885,1,7
13260,7.0,컴퓨터,850,1,8
13259,8.0,컴퓨터,845,5,13
13258,9.0,컴퓨터,800,4,17
13257,10.0,컴퓨터,750,1,18


In [167]:
a[(a['rank'] == 1) & (a['count'] != 1)].groupby('keyword').size().reset_index().sort_values(0, ascending=False)

,keyword,0
0,105,1
797,아이폰se2케이스,1
777,아이폰7 케이스,1
776,아이폰6플러스,1
775,아이폰6중고폰,1
...,...,...
386,다이나핏 집업,1
385,다이나핏 맨투맨,1
384,다미아니목걸이,1
383,다꾸나눔,1


In [177]:
compare_3hr = pd.merge(pre_mean, post_3hr_mean, on='ad_set_id')
compare_3hr['diff'] = compare_3hr['rank_x'] - compare_3hr['rank_y']

In [178]:
compare_3hr.describe()

,ad_set_id,rank_x,rank_y,diff
count,1784.000000,1784.000000,1784.000000,1784.000000
mean,79322.096973,2.289780,2.319307,-0.029526
std,2649.374266,1.888209,1.833153,0.318344
min,63454.000000,1.000000,1.000000,-4.000000
25%,78894.500000,1.142857,1.200000,-0.069085
50%,80035.500000,1.571429,1.666667,0.000000
75%,80746.250000,2.500000,2.600000,0.000000
max,81302.000000,16.000000,14.666667,5.000000


In [132]:
c

,ad_set_id,rank_x,rank_y,diff
0,63454,6.280000,6.080000,0.200000
1,64002,11.657658,11.063063,0.594595
2,64083,2.500000,2.416667,0.083333
3,64113,1.400000,1.600000,-0.200000
4,64209,3.666667,3.625000,0.041667
...,...,...,...,...
1779,81296,1.166667,1.333333,-0.055556
1780,81297,1.000000,1.333333,0.333333
1781,81298,1.250000,1.375000,0.000000
1782,81301,1.666667,1.750000,0.000000


In [129]:
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

q = '''
SELECT trim(k.keyword) as keyword,
       k.bid_price,
       p.id as ad_set_product_id,
       a.id as ad_set_id
FROM ad_set a
JOIN ad_set_product p on a.id = p.ad_set_id
JOIN ad_set_product_bid_keyword k ON k.ad_set_product_id = p.id
WHERE a.status = 3
'''
ad_set6 = pd.read_sql(q, con=mysql)
print('Done getting ad set data')

q = '''
select if(@grouping = tmp.keyword, @rank:=@rank+1, @rank:=1) AS 'rank',
       @grouping:=tmp.keyword as 'keyword',
       tmp.bid_price AS 'bid_price',
       tmp.count
from
(select trim(apk.keyword) AS keyword, apk.bid_price, count(*) AS 'count' from
        ad_set ad
        join ad_set_product ap on ad.id = ap.ad_set_id
        join ad_set_product_bid_keyword apk on apk.ad_set_product_id = ap.id
        where ad.status = 3
          and ap.status = 3
          and apk.active = 1
        group by 1, 2
            order by 1 asc, 2 desc) as tmp,
(select @rank:=0, @grouping:='') as b
group by tmp.keyword, tmp.bid_price
'''
bid_keyword_rank6 = pd.read_sql(q, con=mysql)
print('Done getting bid keyword rank data')

ad_set6['bid_price'] = ad_set6['bid_price'].astype(int)
bid_keyword_rank6['bid_price'] = bid_keyword_rank6['bid_price'].astype(int)
bid_keyword_rank6['rank'] = bid_keyword_rank6['rank'].astype(float)

ad6 = pd.merge(ad_set6, bid_keyword_rank6, on=['keyword', 'bid_price'])
post_18hr_mean = ad6.groupby('ad_set_id', as_index=False)['rank'].mean()

Current Time = 10:11:38
Done getting ad set data
Done getting bid keyword rank data


In [130]:
compare_18hr = pd.merge(pre_mean, post_18hr_mean, on='ad_set_id')
compare_18hr['diff'] = compare_18hr['rank_x'] - compare_18hr['rank_y']

In [131]:
compare_18hr.describe()

,ad_set_id,rank_x,rank_y,diff
count,1435.000000,1435.000000,1435.000000,1435.000000
mean,79360.716376,2.357969,2.398834,-0.040865
std,2752.659128,1.980634,1.771961,0.782327
min,64002.000000,1.000000,1.000000,-2.500000
25%,79198.500000,1.166667,1.281746,-0.328489
50%,80109.000000,1.600000,1.833333,0.000000
75%,80808.000000,2.566964,2.773504,0.000000
max,81302.000000,16.000000,14.500000,8.000000


In [174]:
def get_cum_rank(bid_keyword_rank):
    a = bid_keyword_rank.sort_values(['keyword', 'rank'])
    cum_rank = []
    ks = a.keyword.unique()
    r = 1
    for k in ks:
        b = a[a['keyword'] == k].sort_values('rank')
        cum = b['count'].cumsum().tolist()
        cum_rank += cum
    a['cum_rank'] = cum_rank
    return a

In [175]:
bid_keyword_rank_ = get_cum_rank(bid_keyword_rank)
bid_keyword_rank2_ = get_cum_rank(bid_keyword_rank2)
bid_keyword_rank3_ = get_cum_rank(bid_keyword_rank3)
bid_keyword_rank4_ = get_cum_rank(bid_keyword_rank4)
bid_keyword_rank5_ = get_cum_rank(bid_keyword_rank5)

In [176]:
bid_keyword_rank_['bid_price'] = bid_keyword_rank_['bid_price'].astype(int)
bid_keyword_rank_['rank'] = bid_keyword_rank_['rank'].astype(float)
bid_keyword_rank2_['bid_price'] = bid_keyword_rank2_['bid_price'].astype(int)
bid_keyword_rank2_['rank'] = bid_keyword_rank2_['rank'].astype(float)
bid_keyword_rank3_['bid_price'] = bid_keyword_rank3_['bid_price'].astype(int)
bid_keyword_rank3_['rank'] = bid_keyword_rank3_['rank'].astype(float)
bid_keyword_rank4_['bid_price'] = bid_keyword_rank4_['bid_price'].astype(int)
bid_keyword_rank4_['rank'] = bid_keyword_rank4_['rank'].astype(float)
bid_keyword_rank5_['bid_price'] = bid_keyword_rank5_['bid_price'].astype(int)
bid_keyword_rank5_['rank'] = bid_keyword_rank5_['rank'].astype(float)

In [182]:
ad_ = pd.merge(ad_set, bid_keyword_rank_, on=['keyword', 'bid_price'])
ad5_ = pd.merge(ad_set5, bid_keyword_rank5_, on=['keyword', 'bid_price'])

pre_mean_tmp = ad_.groupby('ad_set_id', as_index=False)['cum_rank'].mean()
post_3hr_mean_tmp = ad5_.groupby('ad_set_id', as_index=False)['cum_rank'].mean()
compare_3hr_tmp = pd.merge(pre_mean_tmp, post_3hr_mean_tmp, on='ad_set_id')
compare_3hr_tmp['diff'] = compare_3hr_tmp['cum_rank_x'] - compare_3hr_tmp['cum_rank_y']

In [186]:
compare_3hr_tmp.describe()

,ad_set_id,cum_rank_x,cum_rank_y,diff
count,1784.000000,1784.000000,1784.000000,1784.000000
mean,79322.096973,5.622654,5.706953,-0.084299
std,2649.374266,6.705736,6.613291,0.943907
min,63454.000000,1.000000,1.000000,-13.000000
25%,78894.500000,1.500000,1.500000,-0.231609
50%,80035.500000,3.000000,3.118056,0.000000
75%,80746.250000,7.016907,7.000000,0.000000
max,81302.000000,60.100000,47.800000,13.000000


In [188]:
pre_product_ad = ad_.groupby('ad_set_product_id', as_index=False)['rank'].mean()
post_product_ad = ad5_.groupby('ad_set_product_id', as_index=False)['rank'].mean()
t_product = pd.merge(pre_product_ad, post_product_ad, on='ad_set_product_id')
t_product['diff'] = t_product['rank_x'] - t_product['rank_y']

In [200]:
ad[ad['ad_set_product_id'] == 219403]

,keyword,bid_price,ad_set_product_id,ad_set_id,rank,count
11656,아이폰12,50,219403,80044,14.0,21
21024,아이폰12 화이트,50,219403,80044,1.0,1


In [202]:
ad5[ad5['ad_set_product_id'] == 219403]

,keyword,bid_price,ad_set_product_id,ad_set_id,rank,count
10984,아이폰12,50,219403,80044,15.0,20


In [201]:
ad[ad['ad_set_product_id'] == 218171]

,keyword,bid_price,ad_set_product_id,ad_set_id,rank,count
1451,s10,50,218171,79555,11.0,15
2479,S105G,50,218171,79555,1.0,16


In [203]:
ad5[ad5['ad_set_product_id'] == 218171]

,keyword,bid_price,ad_set_product_id,ad_set_id,rank,count
6904,s10,50,218171,79555,10.0,16


In [204]:
ad[ad['ad_set_product_id'] == 220792]['']

,keyword,bid_price,ad_set_product_id,ad_set_id,rank,count
5084,바람막이,355,220792,80769,6.0,3
10903,트레이닝복세트,235,220792,80769,2.0,1
10904,나이키,365,220792,80769,12.0,2


In [205]:
ad5[ad5['ad_set_product_id'] == 220792]

,keyword,bid_price,ad_set_product_id,ad_set_id,rank,count
3374,바람막이,355,220792,80769,7.0,2
10079,나이키,365,220792,80769,12.0,1
